In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the dataset
file_path = 'BigBasket Products.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()


,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [15]:
# Checking for missing values
missing_values = data.isnull().sum()

# Checking for duplicate entries
duplicate_entries = data.duplicated().sum()

# Overview of the rating distribution
rating_distribution = data['rating'].describe()

missing_values, duplicate_entries, rating_distribution


(index              0
 product            1
 category           0
 sub_category       0
 brand              1
 sale_price         0
 market_price       0
 type               0
 rating          8626
 description      115
 dtype: int64,
 0,
 count    18929.000000
 mean         3.943410
 std          0.739063
 min          1.000000
 25%          3.700000
 50%          4.100000
 75%          4.300000
 max          5.000000
 Name: rating, dtype: float64)

In [16]:
# Drop rows with missing 'rating' as they are critical for popularity-based recommendations
data.dropna(subset=['rating'], inplace=True)

categorical_features=['category','sub_category','brand']
label_encoders={}

for feature in categorical_features:
  le=LabelEncoder()
  data[feature]=le.fit_transform(data[feature].astype(str))
  label_encoders[feature]=le

# Drop rows with missing 'description' or replace NaN with an empty string
data['description'].fillna('', inplace=True)

# Label encode categorical features
categorical_features = ['category', 'sub_category', 'brand']
encoder = LabelEncoder()

for feature in categorical_features:
    data[feature] = encoder.fit_transform(data[feature].astype(str))

# Process text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['description'])
text_sequences = tokenizer.texts_to_sequences(data['description'])
text_padded = pad_sequences(text_sequences, maxlen=100)  # Choose maxlen based on your data

# Normalize numerical features
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(text_padded, data['rating'], test_size=0.2, random_state=42)


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout

input_dim = text_padded.shape[1]  # This is the maxlen of your text sequences

# Define the encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)

# Define the decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)

# Compile the autoencoder
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Summary of the model
autoencoder.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_5 (Dense)             (None, 128)               12928     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dense_9 (Dense)             (None, 100)               12900     
                                                                 
Total params: 46564 (181.89 KB)
Trainable params: 46564 (18

In [18]:
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)


Epoch 1/50
48/48 [==============================] - 2s 8ms/step - loss: -100013440.0000 - val_loss: -886679552.0000
Epoch 2/50
48/48 [==============================] - 0s 5ms/step - loss: -29273403392.0000 - val_loss: -131525910528.0000
Epoch 3/50
48/48 [==============================] - 0s 5ms/step - loss: -821401944064.0000 - val_loss: -2428650389504.0000
Epoch 4/50
48/48 [==============================] - 0s 5ms/step - loss: -7809781465088.0000 - val_loss: -17563579842560.0000
Epoch 5/50
48/48 [==============================] - 0s 5ms/step - loss: -40386647031808.0000 - val_loss: -75587901194240.0000
Epoch 6/50
48/48 [==============================] - 0s 5ms/step - loss: -142607174860800.0000 - val_loss: -237703320829952.0000
Epoch 7/50
48/48 [==============================] - 0s 6ms/step - loss: -396795721023488.0000 - val_loss: -604267874353152.0000
Epoch 8/50
48/48 [==============================] - 0s 5ms/step - loss: -926201979863040.0000 - val_loss: -1330363032928256.0000
Epoc

In [19]:
# Extract the encoder part of the autoencoder
encoder_model = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-3].output)  # Assuming -3 is the encoded layer

# Generate embeddings for the entire dataset
item_embeddings = encoder_model.predict(text_padded)


592/592 [==============================] - 1s 1ms/step


In [24]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(item_embeddings)

# Function to recommend items based on a given item ID
def recommend_items(item_id, similarity_matrix, items, top_k=5):
    # Get similarity scores for the item
    similarity_scores = list(enumerate(similarity_matrix[item_id]))

    # Sort the items based on the similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top-k most similar items
    similarity_scores = similarity_scores[1:top_k+1]

    # Get the item indices
    item_indices = [i[0] for i in similarity_scores]

    recommend_items=items.iloc[item_indices]

    for feature in categorical_features:
      recommend_items[feature]=label_encoders[feature].inverse_transform(recommend_items[feature])

    # Return the top-k most similar items
    return recommend_items

# Example: Recommend items for the first item in the dataset
recommended_items = recommend_items(6, similarity_matrix, data, top_k=5)
print(recommended_items)


     index                          product                  category  \
12      13  Face Wash - Oil Control, Active          Beauty & Hygiene   
22      23                       Chia Seeds  Foodgrains, Oil & Masala   
46      47  After Shave Splash - Arctic Ice          Beauty & Hygiene   
172    173                Pain Relief - Oil          Beauty & Hygiene   
219    220                Gentle Baby Wipes                 Baby Care   

                sub_category                brand  sale_price  market_price  \
12      Rice & Rice Products             ChingTam      110.00         110.0   
22            Dairy & Cheese  Bhikhaaram Chandmal      120.00         120.0   
46             Indian Mithai            Satinance      459.62         469.0   
172  Fresheners & Repellents               Streax       90.00         100.0   
219     Cookware & Non Stick            SterloMax      699.00         699.0   

                 type  rating  \
12          Face Care   1.000   
22   Other Dry Fruit

<ipython-input-24-1c38dcfebeb1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_items[feature]=label_encoders[feature].inverse_transform(recommend_items[feature])
<ipython-input-24-1c38dcfebeb1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_items[feature]=label_encoders[feature].inverse_transform(recommend_items[feature])
<ipython-input-24-1c38dcfebeb1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [21]:
# Evaluate the model on the test set
test_loss = autoencoder.evaluate(X_test, X_test)
print(f'Test Loss: {test_loss}')


119/119 [==============================] - 0s 2ms/step - loss: -12492653313814167552.0000
Test Loss: -1.2492653313814168e+19
